In [2]:
import os
import pandas as pd
import json
import numpy as np

# Read Json file & PreProcessing

In [3]:
!unzip "../data/data.zip" -d "../data"

unzip:  cannot find or open ../data/data.zip, ../data/data.zip.zip or ../data/data.zip.ZIP.


In [4]:
file_names = os.listdir('../data/datas')
datas = []

for elem in file_names:
    path = os.path.join('../data/datas', elem)
    try:
        with open(path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            datas.append(data)
    except UnicodeDecodeError:
        print(f"Skipping file: {path} (UnicodeDecodeError)")

FileNotFoundError: ignored

In [5]:
datas[0]

NameError: ignored

In [6]:
features = []
for store in datas:
    meta = store['meta']
    annotations = store['annotations']
    for annot in annotations:
        data = {}
        data['food_type'] = annot['menu_information']['food_type']
        data['food_subtype'] = annot['menu_information']['food_subtype']
        data['food_name'] = annot['menu_information']['ko']
        data['food_price'] = annot['menu_information']['price']
        data['ingred'] = annot['menu_information']['ingredients.ko']
        data['date'] = meta['captured_date'][0:4]
        data['store_type'] = meta['store_type']
        data['store_region'] = meta['store_region']
        features.append(data)

NameError: ignored

In [ ]:
df = pd.DataFrame(features)
df

,food_type,food_subtype,food_name,food_price,ingred,date,store_type,store_region
0,한식,구이/적,대패삼겹살,3700,[돼지고기],2022,KF01,DJ
1,한식,탕,동태내장탕,6000,"[야채, 고추, 생선곤이, 동태]",2022,KF01,DJ
2,음료/주류,주류,소주,4000,[알코올],2022,KF01,DJ
3,한식,구이/적,일반삼겹살,4900,[돼지고기],2022,KF01,DJ
4,한식,찌개,양푼이동태찌개,6000,"[야채, 두부, 동태, 쑥갓]",2022,KF01,DJ
...,...,...,...,...,...,...,...,...
1092964,한식,튀김,떡강정,5000,[떡],2022,KF06,IN
1092965,분식/기타,기타,양념치킨,14000,"[닭고기, 밀가루]",2022,KF06,IN
1092966,분식/기타,기타,양념슈프림치킨,14000,[닭고기],2022,KF06,IN
1092967,음료/주류,주류,생맥주,7000,"[보리, 알코올]",2022,KF06,IN


In [ ]:
df.food_type.unique()

array(['한식', '음료/주류', '분식/기타', '중식', '일식', '양식', '외국음식'], dtype=object)

In [ ]:
df.food_subtype.unique()

array(['구이/적', '탕', '주류', '찌개', '볶음', '밥', '면', '음료', '찜', '기타', '국',
       '분식', '전골', '상차림', '만두/떡국', '전', '메인요리', '죽', '회', '튀김', '전채요리',
       '요리', '생채/숙채', '조림', '젓갈', '떡', '장', '김치', '장아찌', '한과'],
      dtype=object)

In [ ]:
df.store_type.unique()

array(['KF01', 'KF04', 'KF06', 'KF02', 'IF01', 'IF03', 'KF03', 'FF01',
       'IF02', 'KF07', 'KF05', 'BF01'], dtype=object)

In [ ]:
df.store_region.unique()

array(['DJ', 'SL', 'KS', 'IN', 'GK', 'CC', 'BS', 'GJ', 'US', 'DG', 'JJ',
       'KW', 'JR'], dtype=object)

In [ ]:
df.sample(5)

,food_type,food_subtype,food_name,food_price,ingred,date,store_type,store_region
480421,한식,기타,왕사발비빔냉면+갈비튀김만두,9900,[],2022,KF02,JR
637680,음료/주류,음료,스프라이트,2000,[탄산],2022,KF04,KS
906054,분식/기타,기타,양념반후라이드,19000,"[닭고기, 밀가루]",2022,KF06,KW
618308,일식,튀김,치즈돈가스,9900,"[돼지고기, 치즈, 계란, 밀가루]",,KF03,CC
249151,한식,찌개,김치찌개,6000,"[채소, 김치]",2022,KF04,SL


In [ ]:
from collections import Counter
ingreds = []
for i in range(len(df)):
    ingreds+=df['ingred'].iloc[i]
count = Counter(ingreds)

In [ ]:
menus = [x[0] for x in count.most_common(50)]

In [ ]:
df = df[df['ingred'].apply(lambda x: all(item in menus for item in x))]
df = df.drop('store_type', axis=1)
df = df.drop('date', axis=1)
df.food_price = df.food_price.astype('int')


In [ ]:
df

,food_type,food_subtype,food_name,food_price,ingred,store_region
0,한식,구이/적,대패삼겹살,3700,[돼지고기],DJ
1,한식,탕,동태내장탕,6000,"[야채, 고추, 생선곤이, 동태]",DJ
2,음료/주류,주류,소주,4000,[알코올],DJ
3,한식,구이/적,일반삼겹살,4900,[돼지고기],DJ
4,한식,찌개,양푼이동태찌개,6000,"[야채, 두부, 동태, 쑥갓]",DJ
...,...,...,...,...,...,...
1092964,한식,튀김,떡강정,5000,[떡],IN
1092965,분식/기타,기타,양념치킨,14000,"[닭고기, 밀가루]",IN
1092966,분식/기타,기타,양념슈프림치킨,14000,[닭고기],IN
1092967,음료/주류,주류,생맥주,7000,"[보리, 알코올]",IN


In [1]:
df['ingred']

NameError: ignored

In [ ]:
df.isnull().sum()

food_type       0
food_subtype    0
food_name       0
food_price      0
ingred          0
store_region    0
dtype: int64

In [ ]:
df.isna().sum()

food_type       0
food_subtype    0
food_name       0
food_price      0
ingred          0
store_region    0
dtype: int64

In [ ]:
df.to_csv("../data/processed_data.csv")

In [ ]:
"생선곤이" in menus

False

In [ ]:
pd.read_csv("../data/processed_data.csv")

,Unnamed: 0,food_type,food_subtype,food_name,food_price,ingred,store_region
0,0,한식,구이/적,대패삼겹살,3700,['돼지고기'],DJ
1,2,음료/주류,주류,소주,4000,['알코올'],DJ
2,3,한식,구이/적,일반삼겹살,4900,['돼지고기'],DJ
3,5,한식,볶음,고추장불고기,4500,['돼지고기'],DJ
4,6,음료/주류,주류,맥주,4000,"['보리', '알코올']",DJ
...,...,...,...,...,...,...,...
732353,1092964,한식,튀김,떡강정,5000,['떡'],IN
732354,1092965,분식/기타,기타,양념치킨,14000,"['닭고기', '밀가루']",IN
732355,1092966,분식/기타,기타,양념슈프림치킨,14000,['닭고기'],IN
732356,1092967,음료/주류,주류,생맥주,7000,"['보리', '알코올']",IN


In [ ]:
new_df

,food_type,food_subtype,food_name,food_price,ingred,store_region
0,한식,구이/적,대패삼겹살,3700,[돼지고기],DJ
1,한식,탕,동태내장탕,6000,"[야채, 고추, 생선곤이, 동태]",DJ
2,음료/주류,주류,소주,4000,[알코올],DJ
3,한식,구이/적,일반삼겹살,4900,[돼지고기],DJ
4,한식,찌개,양푼이동태찌개,6000,"[야채, 두부, 동태, 쑥갓]",DJ
...,...,...,...,...,...,...
1092964,한식,튀김,떡강정,5000,[떡],IN
1092965,분식/기타,기타,양념치킨,14000,"[닭고기, 밀가루]",IN
1092966,분식/기타,기타,양념슈프림치킨,14000,[닭고기],IN
1092967,음료/주류,주류,생맥주,7000,"[보리, 알코올]",IN


# encoding

In [ ]:
food_type_row = df.food_type
food_subtype_row = df.food_subtype

ft_encoded = pd.get_dummies(df.food_type)
fst_encoded = pd.get_dummies(df.food_subtype)
sr_encoded = pd.get_dummies(df.store_region)


In [ ]:
result = pd.concat([ft_encoded, fst_encoded,sr_encoded], axis=1)
result

,분식/기타,양식,외국음식,음료/주류,일식,중식,한식,구이/적,국,기타,...,DJ,GJ,GK,IN,JJ,JR,KS,KW,SL,US
0,0,0,0,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092964,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1092965,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1092966,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1092967,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
ing = df.ingred.to_numpy()

In [ ]:
all_items = set(item for sublist in ing for item in sublist)
all_items = list(all_items)


multi_hot_encoding = pd.DataFrame(columns=all_items, dtype=int)

for data_point in ing:
    multi_hot_vector = np.zeros(len(all_items), dtype=int)
    for item in data_point:
        item_index = all_items.index(item)
        multi_hot_vector[item_index] = 1
    multi_hot_encoding = multi_hot_encoding.append(pd.Series(multi_hot_vector, index=all_items), ignore_index=True)


In [ ]:
multi_hot_encoding.to_csv("../data/ingredients.csv")

# word embedding using Fasttext

In [ ]:
df.food_name.to_list()

['대패삼겹살',
 '소주',
 '일반삼겹살',
 '고추장불고기',
 '맥주',
 '콩나물국밥',
 '차돌박이',
 '청하',
 '물냉면',
 '막걸리',
 '볶음밥',
 '음료',
 '순대정식',
 '편육정식',
 '소뼈맑은해장국',
 '순대국',
 '편육',
 '매콤뚝배기돼지불고기',
 '신얼큰버섯순대국',
 '소주',
 '뚝배기돼지불고기',
 '돼지고기김치찜',
 '순대해장라면',
 '음료',
 '맥주',
 '소한마리',
 '소반마리',
 '돼지한마리',
 '돼지반마리',
 '삼겹살',
 '한우차돌짬뽕라면',
 '차돌초밥용밥',
 '누룽지계란찜',
 '양념반+마늘반치킨',
 '데리반+마늘반',
 '매운양념반+후라이드반',
 '매운양념반+마늘반',
 '후라이드반+마늘반치킨',
 '매운양념치킨',
 '음료수',
 '소주',
 '맥주',
 '들깨칼국수',
 '사리',
 '라면사리',
 '공기밥',
 '판모밀',
 '보쌈정식',
 '개성김치만두전골',
 '해물파전정식',
 '개성고기만두전골',
 '국수사리추가',
 '모둠전',
 '조랭이떡만둣국',
 '해물파전',
 '가래떡만둣국',
 '해물김치전',
 '김치떡만둣국',
 '조랭이떡국',
 '가래떡국',
 '생감자전',
 '개성만둣국',
 '개성김치만둣국',
 '돼지보쌈',
 '개성만두찜',
 '개성김치만두찜',
 '개성버섯만두찜',
 '모둠만두찜',
 '치즈볼',
 '떡산적스테이크',
 '목살덮밥',
 '맛감자튀김',
 '갈비목살덮밥',
 '케이준감자튀김',
 '매콤달달제육덮밥',
 '고추장닭고기덮밥',
 '공기밥',
 '계란후라이',
 '미소된장국',
 '갈비맛목살',
 '김치',
 '간장닭고기',
 '마요네즈소스',
 '매콤달달제육볶음',
 '고추장닭고기',
 '케찹',
 '콜라',
 '제로콜라',
 '로제파스타',
 '복숨아아이스티',
 '토마토파스타',
 '테라',
 '참이슬후레쉬',
 '불낙죽',
 'LA갈비솥밥세트',
 '참치회뱃살',
 '참치회스페샬',
 '복분자',
 '냉면',
 '뒷고기',
 '

In [ ]:
import fasttext.util
import fasttext

model_path = "../model/price_prediction_model.pth"
loaded_model = fasttext.load_model(model_path)
word_vectors = []
for name in names:
    word = loaded_model.get_word_vector(name)
    word_vectors.append(word)

In [ ]:
np.save("../data/wordvec2.npy", np.array(word_vectors))